In [1]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mlxtend.data import loadlocal_mnist
import timeit
import timeit 
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

images_train,labels_train  = loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/train-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/train-labels.idx1-ubyte')
images_test,labels_test= loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/t10k-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/t10k-labels.idx1-ubyte')


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
def classification_report_to_dataframe2(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        print(line)
        p = re.compile("weighted avg")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [3]:
excel_file_name='svm_ppca_' + str(datetime.now().strftime('%Y%m%d%H%m%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df=pd.DataFrame(columns=['Components','Accuracy','Normalized Accuracy',
                         'Time_taken_train','Time_taken_test','Negative Log loss',
                         'Macro_auc','Micro_auc'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [4]:
scaler=StandardScaler()

In [ ]:
ppca=[1,2,5,10,15,20,30,40,50,60,75,80,100,150,200,250,300,325,350,400,450,500,550,600,700,784]
set=[100,200,250,300,500,1000,2000,5000,10000,20000,30000,40000,50000,60000]

for t in set:
    
    for p in ppca:
        #images_train = np.fromfile('train_img' + str(p) + '.dat', dtype='float64')
        #images_test = np.fromfile('test_img' + str(p) + '.dat', dtype='float64')
        #print(np.shape(images_train))
        #images_train = np.reshape(images_train,(-1,p))
        #images_test = np.reshape(images_test,(-1,p))

        scaler.fit(images_train)
        print('For',p)
        # Apply transform to both the training set and the test set.
        train_img = scaler.transform(images_train)
        test_img = scaler.transform(images_test)

        pca = PCA(n_components=p)
        pca.fit(train_img)


        train_img = pca.transform(train_img)
        test_img = pca.transform(test_img)
        svm = LinearSVC(dual=True)
        classifier= CalibratedClassifierCV(svm)
        start=time()
        classifier.fit(train_img,labels_train)
        train_time=time()-start
        print("trained")
        print("prediction started")
        start=time()
        predictions=classifier.predict(test_img)
        test_time=time()-start
        print("predicted")
        accuracy=accuracy_score(predictions,labels_test,normalize=False)
        normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
        log_loss_prediction=classifier.predict_proba(test_img)
        log_loss_val=log_loss(labels_test,log_loss_prediction)
        classification_repo=classification_report(labels_test,predictions)
        print(classification_repo)
        labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
        predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
        micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
        macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")
        cm = confusion_matrix(labels_test,predictions)
        cm_df = pd.DataFrame(data=cm)
        print(micro_auc,macro_auc)
        df_data=[p,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
        df.loc[len(df)]=df_data
        df_classification_report=classification_report_to_dataframe2(classification_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='logistic_cr_')
        book = load_workbook(excel_file_name)
        print("writing")
        with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
            excel_writer.book = book
            #df_classification_report = classification_report_to_dataframe2(classfication_repo)
            df_classification_report.to_excel(excel_writer, sheet_name='svm_ppca_cr_' + str(p))
            cm_df.to_excel(excel_writer, sheet_name='svm_ppca_cr_' + str(p), 
                           startrow=len(df_classification_report)+5)
            excel_writer.save()
            print('w')
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='svm_ppca_cr_')
    excel_writer.save()
    print('w')


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


For 1


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/

trained
prediction started
predicted
              precision    recall  f1-score   support

           0       0.43      0.79      0.56       980
           1       0.56      0.92      0.69      1135
           2       0.22      0.28      0.25      1032
           3       0.17      0.34      0.22      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.17      0.48      0.25      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

   micro avg       0.29      0.29      0.29     10000
   macro avg       0.15      0.28      0.20     10000
weighted avg       0.16      0.29      0.21     10000

0.607833333333 0.601019719209
weighted avg       0.16      0.29      0.21     10000


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


...

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


w


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


For 250


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
